In [68]:
import sys
sys.path.append('../../common')
import utils
import importlib
importlib.reload(utils)

import time
import os
import logging
import cv2

import pynput.keyboard as kb
import pynput.mouse as m
import numpy as np

from glob import glob
from datetime import datetime
from mss import mss
from PIL import Image
from enum import Enum

if os.name == "nt":
  # windows: Microsoft edge, win+<-, my desktop, monitor (1920x1080)
  io = utils.IOHandler(offset=(389, 230), game_dims=(600, 450), verbose=True)
elif os.name == "posix":
  # MacOS: Desktop app, top left, scaled more space (2048x1280), MBP
  io = utils.IOHandler(offset=(1081, 675), game_dims=(1200,900), verbose=True)

In [69]:
class Screen(Enum):
    GAME_OVER = "game_over"
    HOME = "home"
    GREAT_PLAY = "great_play"
    SCORE_EXCEEDS = "score_exceeds"
    IN_GAME = "in_game"
    GET_READY = "get_ready"
    GREAT_JOB = "great_job"
    OUT_OF_TIME = "out_of_time"
    ERROR_OCCURRED = "error_occurred"

SCREEN_TYPES = {
    Screen.GAME_OVER:       [680, 380, [246, 196, 195]], # pink pig
    Screen.HOME:            [110,  30, [248, 205,  85]], # dark yellow diagonal stripe
    Screen.GREAT_PLAY:      [290, 670, [172,  68,  59]], # dark red gift box
    Screen.SCORE_EXCEEDS:   [330, 465, [ 31,  14, 137]], # webkinz logo outline
    Screen.IN_GAME:         [1055, 65, [179, 108, 247]], # purple music note
    Screen.GET_READY:       [760, 495, [  5,  13, 141]], # 'y' in 'Get Ready!'
    Screen.GREAT_JOB:       [550, 480, [  5,  13, 141]], # 'a' in 'Great Job!'
    Screen.OUT_OF_TIME:     [395, 435, [  5,  13, 141]], # 'R' in 'Ran Out of Time!'
    Screen.ERROR_OCCURRED:  [275, 420, [ 31,  14, 137]], # webkinz 'w' outline
}

LEVEL_INFO = {
    1:	{'offset': [255, 161], 'dims': [694,  692]},
    2:	{'offset': [ 83, 161], 'dims': [1038, 692]},
    3:	{'offset': [254, 156], 'dims': [695,  694]},
    4:	{'offset': [130, 156], 'dims': [924,  694]},
    5:	{'offset': [244, 156], 'dims': [691,  690]},
    6:	{'offset': [174, 156], 'dims': [864,  690]},
    7:	{'offset': [278, 153], 'dims': [694,  692]},
    8:	{'offset': [191, 153], 'dims': [832,  692]},
    9:	{'offset': [290, 165], 'dims': [687,  686]},
    10:	{'offset': [203, 165], 'dims': [801,  686]},
    11:	{'offset': [231, 173], 'dims': [689,  688]},
    12:	{'offset': [211, 173], 'dims': [787,  688]},
    13:	{'offset': [237, 179], 'dims': [691,  690]},
    14:	{'offset': [217, 179], 'dims': [777,  690]},
    15:	{'offset': [242, 184], 'dims': [687,  686]},
    16: {'offset': [222, 184], 'dims': [763,  686]},
    17: {'offset': [246, 188], 'dims': [683,  682]}
}

SYMBOLS = {
    "flipped": f"\033[48;5;21m❓\033[0m", 
    "empty": f"\033[48;5;239m  \033[0m"
}

EMOJIS = {
    "arte": "💎",
    "bunny": "🐰",
    "cat": "😸",
    "cow": "🐮",
    "dog": "🐶",
    "elephant": "🐘",
    "empty": "⬜️",
    "flipped": "🟦",
    "frog": "🐸",
    "gorilla": "🦍",
    "hippo": "🦛",
    "horse": "🐴",
    "lion": "🦁",
    "monkey": "🐵",
    "panda": "🐼",
    "pig": "🐷",
    "poodle": "🐩",
    "pug": "💀",
    "quack": "🦆",
    "unicorn": "🦄",
    "wacky": "⭐️"
}

BACKGROUNDS = {
    "pink": {"rgb": [215, 163, 247], "code": 171},
    "green": {"rgb": [203, 250, 169], "code": 77},
    "blue": {"rgb": [121, 207, 249], "code": 33},
    "yellow": {"rgb": [255, 253, 84], "code": 220},
}

# TODO: add these to utils instead?
def numpy_to_image(arr):
    return Image.fromarray(arr.swapaxes(0, 1).astype(np.uint8), "RGB")

def image_to_numpy(img):
    return np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]

def image_to_cv2(img):
    img = np.uint8(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def cv2_to_image(img):
    img = img.astype(np.uint8)
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def cv2_to_numpy(img):
    return image_to_numpy(cv2_to_image(img))

def numpy_to_cv2(arr):
    return image_to_cv2(numpy_to_image(arr))


def get_best_mask(target, masks):
    best_dist = -1
    best_idx = 0
    for i, mask in enumerate(masks):
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_idx = i
    return best_idx, best_dist

def get_best_mask_dict(target, masks):
    best_dist = -1
    best_key = 0
    for key, mask in masks.items():
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_key = key
    return best_key, best_dist

def calc_image_dist(A, B):
    return np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))

def get_score(img = None):
    x,y = 195,29
    w,h = 24,35
    num_digits = 5
    
    if img is None:
        score_img = io.capture_portion(x, y, w*num_digits, h)
    else:
        score_img = img[x:x+w*num_digits,y:y+h,:]

    score_digits = []
    for i in range(num_digits):
        digit = get_best_mask(score_img[w*i:w*(i+1)], score_masks)[0]
        if digit == len(score_masks) - 1:
            digit = ''
        score_digits.append(digit)
    return int('0'+''.join(map(str, score_digits)))

def get_level(img = None):
    x,y = 736,29
    w,h = 24,35
    num_digits = 2

    if img is None:
        level_img = io.capture_portion(x, y, w*num_digits, h)
    else:
        level_img = img[x:x+w*num_digits,y:y+h,:]

    level_digits = []
    for i in range(num_digits):
        digit = get_best_mask(level_img[w*i:w*(i+1)], score_masks)[0]
        if digit == len(score_masks) - 1:
            digit = ''
        level_digits.append(digit)
    return int('0'+''.join(map(str, level_digits)))

def get_time(img = None):
    x,y = 466,31
    w,h = 22,32
    num_digits = 3

    if img is None:
        time_img = io.capture_portion(x, y, w*num_digits, h)
    else:
        time_img = img[x:x+w*num_digits,y:y+h,:]

    time_digits = []
    for i in range(num_digits):
        digit = get_best_mask(time_img[w*i:w*(i+1)], time_masks)[0]
        if digit == len(time_masks) - 1:
            digit = ''
        else:
            digit = digit % 10
        time_digits.append(digit)
    return int('0'+''.join(map(str, time_digits)))

def get_money(img = None):
    w,h = 24,35
    y = 29

    x = 901
    num_digits = 4

    if img is None:
        money_img = io.capture_portion(x, y, w*num_digits, h)
    else:
        money_img = img[x:x+w*num_digits,y:y+h,:]

    best_guess = None
    best_error = 1000

    money_digits = []
    error_1 = 0
    for i in range(num_digits):
        digit, error = get_best_mask(money_img[w*i:w*(i+1)], score_masks)
        if digit == len(score_masks) - 1:
            digit = ''
        money_digits.append(digit)
        error_1 += error
    money_digits = [x for x in money_digits if x != '']
    guess_1 = int('0'+''.join(map(str, money_digits)))

    num_digits -= 1

    money_digits = []
    error_2 = 0
    for i in range(num_digits):
        digit, error = get_best_mask(money_img[w*i+w//2:w*(i+1)+w//2], score_masks)
        if digit == len(score_masks) - 1:
            digit = ''
        money_digits.append(digit)
        error_2 += error
    money_digits = [x for x in money_digits if x != '']
    guess_2 = int('0'+''.join(map(str, money_digits)))

    if error_1 < error_2:
        return guess_1
    else:
        return guess_2


def get_level_cards(level):
    return np.array([min(10, 2 + level // 2), min(10, 2 + (level - 1) // 2)])

def get_level_board_offset(level):
    level = min(level, 17)
    return np.array(LEVEL_INFO[level]['offset'])

def get_level_board_dimensions(level):
    level = min(level, 17)
    return np.array(LEVEL_INFO[level]['dims'])

def click_card(level, idx):
    cards = get_level_cards(level)
    offset = get_level_board_offset(level)
    board = get_level_board_dimensions(level)
    pos = offset + (idx + 0.5) / cards * board
    io.click_mouse(pos[0], pos[1])

def capture_card(level, idx, rsize=None, asize=None, img=None, diff=False):
    cards = get_level_cards(level)
    offset = get_level_board_offset(level)
    board = get_level_board_dimensions(level)
    if asize is None:
        if rsize is None:
            size = board / cards
        else:
            size = np.array([rsize, rsize]) * board / cards
    else:
        size = np.array([asize, asize])
    pos = offset + (idx + 0.5) / cards * board - size / 2
    pos[0], pos[1], size[0], size[1] = int(pos[0]), int(pos[1]), int(size[0]), int(size[1])
    pos, size = pos.astype(int), size.astype(int)
    if img is not None:
        card = img[pos[0]:pos[0]+size[0],pos[1]:pos[1]+size[1],:]
    else:
        card = io.capture_portion(pos[0], pos[1], size[0], size[1])
    if diff:
        return subtract(card, bg[pos[0]:pos[0]+size[0],pos[1]:pos[1]+size[1],:])
    return card

def subtract(A, B, atol=1):
    return np.stack((np.any(np.logical_not(np.isclose(A, B, rtol=0.0, atol=atol)), axis=2),)*3, axis=-1)*A

def avg_color(img):
    return np.mean(img, axis=(0, 1))

def decode_board(level, img = None):
    if img is None:
        img = io.capture_screen()
    cards = get_level_cards(level)
    board = np.empty(shape=[cards[0], cards[1]], dtype=object)
    diff = subtract(img, bg)
    for i in range(cards[0]):
        for j in range(cards[1]):
            card = capture_card(level, np.array([i, j]), rsize = 1.0, asize = None, img = diff)
            card = crop_card(card)
            mini = minify_card(card)
            key, dist = get_best_mask_dict(mini, card_masks)
            cutoff = 120
            if dist > cutoff:
                print(f"WARNING: distance from ({cards[0]},{cards[1]}) to '{key}' = {dist} > {cutoff}")
            board[i,j] = key
    return board

def parse_card_after_flip(level, idx, rsize=1.0):
    prev = None
    card = None
    start = time.time()
    while True:
        card = capture_card(level, idx, rsize=rsize, asize=None, img=None, diff=True)
        mini = minify_card(card)
        key, dist = get_best_mask_dict(mini, card_masks)
        if (card == prev).all(): # If card done moving, return type
            return key
        duration = time.time() - start
        if key == 'empty' and dist < 120 and duration > 0.2: # Wait for card to flip mostly & check if mostly empty
            return key
        prev = card
        time.sleep(0.1)

def get_dominant_color(arr):
    img = numpy_to_image(arr)
    img.convert("RGB")
    img.resize((1, 1), resample=0)
    arr = image_to_numpy(img)
    return arr[0,0,:]

def find_known_pair_if_exists(board):
    unmatched = {}
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card != "flipped" and card != "empty":
                if card in unmatched:
                    match = unmatched.pop(card)
                    return np.array([match, [i,j]])
                else:
                    unmatched[card] = np.array([i,j])
    return None

def find_unknown_card_index(board):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card == "flipped":
                return np.array([i,j])
    return None

def count_remaining_matches(board):
    count = 0
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card != "empty":
                count += 1
    return count // 2

def find_partner_index_if_exists(board, other):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            if (np.array([i,j]) == other).all():
                continue
            card = board[i,j]
            if card == board[other[0], other[1]]:
                return np.array([i,j])
    return None

def print_board(board):
    global EMOJIS
    global BACKGROUNDS
    global SYMBOLS
    output = ""
    for j in range(0, board.shape[1]):
        for i in range(0, board.shape[0]):
            card = board[i,j]
            if card not in SYMBOLS:
                animal, color = card.split("_")
                SYMBOLS[card] = f"\033[48;5;{BACKGROUNDS[color]['code']}m{EMOJIS[animal]}\033[0m" # TODO: can get rid of SYMBOLS?
            output += SYMBOLS[card]
        output += "\n"
    print(output)

# TODO: write this somewhere so don't have to calc each time
def calculate_level_info(img=None):
    if img is None:
        img = io.capture_screen()
    diff = subtract(img, bg, atol=1)[:,100:,:]   # ignore first 100 lines
    img = np.any(np.not_equal(diff, np.zeros([3])), axis=-1)
    rows, cols = np.any(img, axis=1), np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    x,y,w,h = ymin, xmin+100, ymax-ymin+1, xmax-xmin+1
    return {'offset': [x,y], 'dims': [w,h]}

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]], count[order[::-1]]

def my_color_hash(img):
    colors, freqs = palette(img)
    hsh = set()
    i = 0
    num_colors = 3
    while len(hsh) < num_colors:
        if i > colors.shape[0] - 1:
            break
        else:
            color = colors[i,:]
            freq = freqs[i] / (img.shape[0]*img.shape[1])
            if freq < 0.02:
                i += 1
                continue
            # if (color==np.array([121,207,249])).all() or (color==np.array([0,25,214])).all():  # ignore light & dark blue
            if (color==np.array([0,25,214])).all():  # ignore dark blue
                i += 1
                continue
            hsh.add("%02x%02x%02x" % (colors[i,0], colors[i,1], colors[i,2]))
            i += 1
    return frozenset(hsh)

def init_board(level):
    cards = get_level_cards(level)
    board = np.empty(shape=[cards[0], cards[1]], dtype=object)
    board.fill("flipped")
    if cards[0] % 2 == 1 and cards[1] % 2 == 1:
        board[cards[0]//2, cards[1]//2] = "empty"
    return board

def minify_card(img):
    return image_to_numpy(numpy_to_image(img).resize([63,63], Image.BILINEAR))

def test_number_parser():
    digit_tests = [
        {'path': 'saved/test_big.png', 'expected': {'score': 47859, 'time': 130, 'level': 24, 'money': 1399}},
        {'path': 'saved/1625982666.8275049.png', 'expected': {'score': 0, 'time': 4, 'level': 1, 'money': 0}},
        {'path': 'saved/1625982702.78526.png', 'expected': {'score': 22, 'time': 7, 'level': 2, 'money': 1}},
        {'path': 'saved/1625982713.5493178.png', 'expected': {'score': 67, 'time': 18, 'level': 3, 'money': 2}},
        {'path': 'saved/1625982729.036376.png', 'expected': {'score': 137, 'time': 6, 'level': 4, 'money': 4}},
        {'path': 'saved/1625982736.947087.png', 'expected': {'score': 150, 'time': 19, 'level': 4, 'money': 5}},
        {'path': 'saved/1625982756.895236.png', 'expected': {'score': 231, 'time': 5, 'level': 5, 'money': 7}},
        {'path': 'saved/1625982798.058444.png', 'expected': {'score': 368, 'time': 1, 'level': 6, 'money': 10}},
        {'path': 'saved/1625982837.043364.png', 'expected': {'score': 428, 'time': 8, 'level': 6, 'money': 13}},
        {'path': 'saved/1625982999.3064802.png', 'expected': {'score': 595, 'time': 11, 'level': 7, 'money': 17}}
    ]
    for i, test in enumerate(digit_tests):
        img = io.load_image(test['path'])
        actual = {'score': get_score(img), 'time': get_time(img), 'level': get_level(img), 'money': get_money(img)}
        if actual != test['expected']:
            print(f"Failed test #{i} ({test['path']}):\n\tACTUAL:\t\t{actual}\n\tEXPECTED:\t{test['expected']}")
            
def crop_card(img):
    mask = np.logical_or(
        np.all(np.equal(img, BACKGROUNDS['pink']['rgb']), axis=-1),
        np.logical_or(
            np.all(np.equal(img, BACKGROUNDS['green']['rgb']), axis=-1),
            np.logical_or(
                np.all(np.equal(img, BACKGROUNDS['yellow']['rgb']), axis=-1),
                np.all(np.equal(img, BACKGROUNDS['blue']['rgb']), axis=-1)
            )
        )
    )
    if (mask == np.zeros([1])).all():
        return img
    rows, cols = np.any(mask, axis=1), np.any(mask, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    x,y,w,h = ymin, xmin, ymax-ymin+1, xmax-xmin+1
    w = h = max(w, h)
    return img[x:x+w, y:y+h, :]

def on_press(key):
    global kill_program
    if key == kb.Key.esc:
        print("Pressed escape to quit")
        kill_program = True
        return False

def log_level_info(msg, details):
    global prev_score, prev_time, prev_level, prev_money, logger
    img = io.capture_screen()
    curr_score, curr_time, curr_level, curr_money = get_score(img), get_time(img), get_level(img), get_money(img)
    score_inc, time_inc, level_inc, money_inc = curr_score - prev_score, curr_time - prev_time, curr_level - prev_level, curr_money - prev_money
    prev_score, prev_time, prev_level, prev_money = curr_score, curr_time, curr_level, curr_money
    logger.debug("\t".join([str(x) for x in [msg, details, curr_score, curr_time, curr_level, curr_money, score_inc, time_inc, level_inc, money_inc]]))
    return curr_score, curr_time, curr_level, curr_money

In [70]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)
fh = logging.FileHandler("wacky.log")
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

game_start = None

time_masks = []
for i in range(10):
    img = Image.open('time/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    time_masks.append(arr)
for i in range(6):
    img = Image.open('time/mask_' + str(i) + "r.png")
    arr = image_to_numpy(img)
    time_masks.append(arr)
img = Image.open('time/mask_blank.png')
arr = image_to_numpy(img)
time_masks.append(arr)

score_masks = []
for i in range(10):
    img = Image.open('score/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    score_masks.append(arr)
img = Image.open('score/mask_blank.png')
arr = image_to_numpy(img)
score_masks.append(arr)

card_masks = {}
for filename in glob('cards/mini/*.png'):
    card = crop_card(io.load_image(filename))
    mini = minify_card(card)
    card_masks[filename[11:-4]] = mini

bg = io.load_image("background.png")

listener = kb.Listener(on_press=on_press)
listener.start()

MAX_SCORE = 30_000 # TODO: it doesn't stop due to time bonus

io.click_mouse(0,0)

# TODO: move to utils?
def is_screen_type(screen_type, image=None):
    x, y, color = SCREEN_TYPES[screen_type]
    return io.is_given_screen(x, y, color, delta=10, image=image)

# TODO: move to utils?
def calc_screen_types(image=None):
    return {screen_type for screen_type in SCREEN_TYPES.keys() if is_screen_type(screen_type, image)}

def get_into_game():
  global kill_program
  # Get to the main play screen
  while True:
    if kill_program:
      break
    game_img = io.capture_screen()
    screen_types = calc_screen_types(game_img)
    print(screen_types)

    if Screen.GET_READY in screen_types:
        pass
    elif Screen.GREAT_JOB in screen_types:
        pass
    elif Screen.OUT_OF_TIME in screen_types:
        pass # No exit button
    elif Screen.SCORE_EXCEEDS in screen_types:
        io.click_mouse(670, 520, delays=[0.3,0,0.3])
    elif Screen.ERROR_OCCURRED in screen_types:
        io.click_mouse(700, 560, delays=[0.3,0,0.3])
    elif Screen.GREAT_PLAY in screen_types:
        io.click_mouse(470, 980, delays=[0.3,0,0.3])
    elif Screen.HOME in screen_types:
        io.click_mouse(460, 690, delays=[0.3,0,0.3]) # '1 Player' button
    elif Screen.GAME_OVER in screen_types:
        io.click_mouse(250, 700,  delays=[0.3,0,0.3]) # 'Replay' button
    elif Screen.IN_GAME in screen_types:
        return

    time.sleep(0.2)

kill_program = False
while not kill_program:
    prev_score, prev_time, prev_level, prev_money = 0,0,0,0

    get_into_game()

    game_img = io.capture_screen()
    level = get_level(game_img)
    print(f"LEVEL: {level}")

    # io.save_image(game_img, f'no-commit/saved/capture/level-{level}-{datetime.now()}.png')

    board = decode_board(level, game_img)
    print_board(board)

    # This loop plays through the current level, by repeatedly flipping, remembering, submitting pairs
    while not kill_program:
        curr_score = get_score()
        if curr_score + (level + 9) > MAX_SCORE:
            log_level_info("MAX SCORE", curr_score)
            io.click_mouse(1160, 45)
            time.sleep(0.5)
            break # Quit this game if next pair would go over the score cap
        curr_time = get_time()

        print_board(board)
        known_pair = find_known_pair_if_exists(board)
        # Flip a known pair if there is one
        if known_pair is not None:
            curr = known_pair[0]
            click_card(level, curr)
            prev = curr
            curr = known_pair[1]
            click_card(level, curr)
            # time.sleep(0.1)
            log_level_info("KNOWN PAIR", board[curr[0], curr[1]])
            print("0: Known pair:", known_pair)
            board[curr[0], curr[1]] = "empty"
            board[prev[0], prev[1]] = "empty"
            print("\tMark empty ^:", known_pair)

            print_board(board)
            continue # Continue with this level after a known pair

        prev = None
        curr = None
        unknown = find_unknown_card_index(board)
        # If no unknown cards, level is done, wait for it
        if unknown is None:
            print("No more matches found")
            log_level_info("LEVEL COMPLETE", level)
            time.sleep(0.5)
            break # Move on to next level if no more matches/unknowns

        # Click first unknown card
        curr = unknown
        click_card(level, curr)
        flipped_card = parse_card_after_flip(level, curr)
        board[curr[0], curr[1]] = flipped_card
        print("1: Unknown ", curr, board[curr[0], curr[1]])

        # Click the 1st card'ss partner, if we know it
        partner = find_partner_index_if_exists(board, curr)
        if partner is not None:
            prev = curr
            curr = partner
            click_card(level, curr)
            # time.sleep(0.1)
            log_level_info("PARTNER FOUND", board[curr[0], curr[1]])
            print("2: Known Partner ", curr)
            board[curr[0], curr[1]] = "empty"
            board[prev[0], prev[1]] = "empty"
            print("\tMatch, mark empty ^:", curr, prev)

            print_board(board)
            continue # Continue with this level after finding a partner
            
        # Click a 2nd unknown
        unknown = find_unknown_card_index(board)
        if unknown is None:
            print("WARNING: No more unknowns to match with flipped card. Danger Will Robinson.")
            break # Quit the game if we don't have any unknowns to match with first card
        prev = curr
        curr = unknown

        click_card(level, curr)
        flipped_card = parse_card_after_flip(level, curr)
        board[curr[0], curr[1]] = flipped_card
        print("2: Unknown ", curr, board[curr[0], curr[1]])

        # Mark empty if we found an incidental match
        if board[curr[0], curr[1]] == "empty":
            log_level_info("INCIDENTAL MATCH", board[prev[0],prev[1]])
            board[prev[0], prev[1]] = "empty"
        print("\tIncidental, mark empty ^:", curr, prev)

        print_board(board)
        continue # Continue with this level after an incidental match

    log_level_info("END OF GAME", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))
log_level_info("BOT TERMINATED", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))

{<Screen.ERROR_OCCURRED: 'error_occurred'>}
{<Screen.GAME_OVER: 'game_over'>}
{<Screen.HOME: 'home'>}
{<Screen.IN_GAME: 'in_game'>}
LEVEL: 1
❓❓
❓❓

❓❓
❓❓

1: Unknown  [0 0] arte_blue
2: Unknown  [0 1] wacky_blue
	Incidental, mark empty ^: [0 1] [0 0]
💎❓
⭐️❓

💎❓
⭐️❓

1: Unknown  [1 0] arte_blue


PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0
PARTNER FOUND	arte_blue	0	6	1	0	0	6	1	0


2: Known Partner  [0 0]
	Match, mark empty ^: [0 0] [1 0]
    
⭐️❓

    
⭐️❓

1: Unknown  [1 1] wacky_blue


PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	0	10	-1	0	0
PARTNER FOUND	wacky_blue	10	5	1	

2: Known Partner  [0 1]
	Match, mark empty ^: [0 1] [1 1]
    
    

    
    

No more matches found


...

KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	dog_green	7506	150	14	222	23	-3	0	1
KNOWN PAIR	do

0: Known pair: [[3 6]
 [6 0]]
	Mark empty ^: [[3 6]
 [6 0]]
  🐷      🦛  ❓❓
💀🐘      🐰❓❓❓
  🐴  🐼🐮  ❓❓❓
    ⭐️🐩  😸❓❓❓
🐵  🐸🐴  💎❓❓❓
    🦍  🐷  ❓❓❓
🦁      🐵  ❓❓❓
  🐮      💀❓❓❓

  🐷      🦛  ❓❓
💀🐘      🐰❓❓❓
  🐴  🐼🐮  ❓❓❓
    ⭐️🐩  😸❓❓❓
🐵  🐸🐴  💎❓❓❓
    🦍  🐷  ❓❓❓
🦁      🐵  ❓❓❓
  🐮      💀❓❓❓

1: Unknown  [6 1] lion_green
Pressed escape to quit


PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	148	14	223	23	-2	0	1
PARTNER FOUND	lion_green	7529	1

2: Known Partner  [0 6]
	Match, mark empty ^: [0 6] [6 1]
  🐷      🦛  ❓❓
💀🐘      🐰  ❓❓
  🐴  🐼🐮  ❓❓❓
    ⭐️🐩  😸❓❓❓
🐵  🐸🐴  💎❓❓❓
    🦍  🐷  ❓❓❓
        🐵  ❓❓❓
  🐮      💀❓❓❓



(7552, 148, 14, 223)